## Import the libraries

In [95]:
import pydicom
import numpy as np
from PIL import Image
import os

In [8]:
dicom_file=pydicom.dcmread("data/slice.IMA")

In [9]:
dicom_file

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 192
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.45508.30000019041301312660900000455
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.3.12.2.1107.5.1.4
(0002,0013) Implementation Version Name         SH: 'SIEMENS_S5VB42A'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'LOCALIZER', 'CT_SOM5 TOP']
(0008,0016) SOP Class UID                       UI: CT Image Storage
(0008,0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.45508.30000019041301312660900000455
(0008,0020) Study Date               

In [10]:
dicom_file.Manufacturer

'SIEMENS'

In [11]:
dicom_array = dicom_file.pixel_array

In [10]:
dicom_array.shape

(512, 512)

## Show Image

In [12]:
type(dicom_array)

numpy.ndarray

In [13]:
np.unique(dicom_array)

array([   0,  782,  786,  789,  792,  799,  800,  802,  805,  806,  813,
        814,  820,  825,  829,  830,  832,  833,  835,  836,  837,  838,
        840,  841,  842,  843,  844,  845,  847,  848,  849,  850,  852,
        853,  854,  855,  856,  857,  858,  859,  860,  861,  862,  863,
        864,  865,  866,  867,  868,  869,  870,  871,  872,  873,  874,
        875,  876,  877,  878,  879,  880,  881,  882,  883,  884,  885,
        886,  887,  888,  889,  890,  891,  892,  893,  894,  895,  896,
        897,  898,  899,  900,  901,  902,  903,  904,  905,  906,  907,
        908,  909,  910,  911,  912,  913,  914,  915,  916,  917,  918,
        919,  920,  921,  922,  923,  924,  925,  926,  927,  928,  929,
        930,  931,  932,  933,  934,  935,  936,  937,  938,  939,  940,
        941,  942,  943,  944,  945,  946,  947,  948,  949,  950,  951,
        952,  953,  954,  955,  956,  957,  958,  959,  960,  961,  962,
        963,  964,  965,  966,  967,  968,  969,  9

In [45]:
float_dicom_array = dicom_array.astype(float)
positive_dicom_array = np.maximum(float_dicom_array,0)

In [48]:
normalized_dicom_array = positive_dicom_array/positive_dicom_array.max()
normalized_dicom_array*=255.0
uint8_image=np.uint8(normalized_dicom_array)

In [22]:
# np.unique(normalized_dicom_array)

In [49]:
pillow_image = Image.fromarray(uint8_image)
pillow_image.show()

In [62]:
def normalize_visualize_dicom_1 (dcm_file, show=False):
    dicom_file=pydicom.dcmread(dcm_file)
    dicom_array = dicom_file.pixel_array.astype(float)
    normalized_dicom_array = ((np.maximum(float_dicom_array,0)) / dicom_array.max()) * 255.0
    uint8_image=np.uint8(normalized_dicom_array)

    if show == True:
        pillow_image = Image.fromarray(uint8_image)
        pillow_image.show()

    return uint8_image

In [65]:
image_1 = normalize_visualize_dicom_1('data/slice.IMA', show=True)

## Show Image 2

In [56]:
hounsfield_min = np.min(dicom_array)
hounsfield_max = np.max(dicom_array)
hounsfield_range = hounsfield_max - hounsfield_min

In [57]:
dicom_array[dicom_array < hounsfield_min] = hounsfield_min
dicom_array[dicom_array > hounsfield_max] = hounsfield_max
normalized_array_2 = (dicom_array - hounsfield_min) / hounsfield_range
normalized_array_2 *= 255
uint8_image_2 = np.uint8(normalized_array_2)

In [58]:
pillow_image = Image.fromarray(uint8_image_2)
pillow_image.show()

In [73]:
def normalize_visualize_dicom_2 (dcm_file, max_v = None, min_v = None, show=True):
    dicom_file=pydicom.dcmread(dcm_file)
    dicom_array = dicom_file.pixel_array

    if max_v is not None: hounsfield_max = max_v
    else: hounsfield_max = np.max(dicom_array)

    if min_v is not None: hounsfield_min = min_v
    else: hounsfield_min = np.min(dicom_array)

    hounsfield_range = hounsfield_max - hounsfield_min

    dicom_array[dicom_array < hounsfield_min] = hounsfield_min
    dicom_array[dicom_array > hounsfield_max] = hounsfield_max
    normalized_array = ((dicom_array - hounsfield_min) / hounsfield_range)*255
    uint8_image = np.uint8(normalized_array)

    if show:
        pillow_image = Image.fromarray(uint8_image)
        pillow_image.show()

    return uint8_image


    

In [78]:
image_2 = normalize_visualize_dicom_2('data/slice.IMA', show=True)

## Anonymize a dicom file


In [79]:
dicom_file.PatientName = 'Aramis'
dicom_file.PatientBirthDate = '01.01.1999'
dicom_file.PatientSex = 'F'
dicom_file.StudyID = ''

d:\anaconda\envs\dicom-env\Lib\site-packages\pydicom\valuerep.py:440: UserWarning: Invalid value for VR DA: '01.01.1999'.
  warn_and_log(msg)


In [80]:
dicom_file.save_as('data/anonymized_slice.IMA')

In [81]:
new_dicom_file = pydicom.dcmread('data/anonymized_slice.IMA')
new_dicom_file.PatientName

'Aramis'

In [83]:
def anonymize_dicom(dcm_file, pn=None, pb=None, save_path=None):
    dicom_file = pydicom(dcm_file)

    if pn:
        dicom_file.PatientName = pn

    else: dicom_file.PatientName = ''

    if pb: dicom_file.PatientBirthDate = pb
    else: dicom_file.PatientBirthDate = ''

    if save_path:
        dicom_file.save_as(save_path)

    return dicom_file



## Changing Tags


In [84]:
def change_tags(dcm_file, ImageType=None, SOPClassUID=None, SOPInstanceUID=None, StudyDate=None,
                SeriesDate=None, ContentDate=None, StudyTime=None, SeriesTime=None,
                ContentTime=None, AccessionNumber=None, Modality=None, Manufacturer=None,
                ReferringPhysicianName=None, PatientName=None, PatientID=None, PatientBirthDate=None,
                PatientSex=None, SliceThickness=None, PatientPosition=None, StudyInstanceUID=None,
                SeriesInstanceUID=None, StudyID=None, SeriesNumber=None, InstanceNumber=None,
                ImagePositionPatient=None, ImageOrientationPatient=None, FrameOfReferenceUID=None,
                PositionReferenceIndicator=None, SamplesPerPixel=None, PhotometricInterpretation=None,
                Rows=None, Columns=None, PixelSpacing=None, BitsAllocated=None, BitsStored=None,
                HighBit=None, PixelRepresentation=None, WindowCenter=None, WindowWidth=None, 
                RescaleIntercept=None, RescaleSlope=None, RescaleType=None, PixelData=None):
    
    
    dicom_file_data = pydicom.dcmread(dcm_file)

    if ImageType: dicom_file_data.ImageType = ImageType
    if SOPClassUID: dicom_file_data.SOPClassUID = SOPClassUID
    if SOPInstanceUID: dicom_file_data.SOPInstanceUID = SOPInstanceUID
    if StudyDate: dicom_file_data.StudyDate = StudyDate
    if SeriesDate: dicom_file_data.SeriesDate = SeriesDate
    if ContentDate: dicom_file_data.ContentDate = ContentDate
    if AccessionNumber: dicom_file_data.AccessionNumber = AccessionNumber
    if Modality: dicom_file_data.Modality = Modality
    if Manufacturer: dicom_file_data.Manufacturer = Manufacturer
    if ReferringPhysicianName: dicom_file_data.ReferringPhysicianName = ReferringPhysicianName
    if PatientName: dicom_file_data.PatientName = PatientName
    if PatientID: dicom_file_data.PatientID = PatientID
    if PatientBirthDate: dicom_file_data.PatientBirthDate = PatientBirthDate
    if PatientSex: dicom_file_data.PatientSex = PatientSex
    if SliceThickness: dicom_file_data.SliceThickness = SliceThickness
    if PatientPosition: dicom_file_data.PatientPosition = PatientPosition
    if StudyInstanceUID: dicom_file_data.StudyInstanceUID = StudyInstanceUID
    if SeriesInstanceUID: dicom_file_data.SeriesInstanceUID = SeriesInstanceUID
    if StudyID: dicom_file_data.StudyID = StudyID
    if SeriesNumber: dicom_file_data.SeriesNumber = SeriesNumber
    if InstanceNumber: dicom_file_data.InstanceNumber = InstanceNumber
    if ImagePositionPatient: dicom_file_data.ImagePositionPatient = ImagePositionPatient
    if ImageOrientationPatient: dicom_file_data.ImageOrientationPatient = ImageOrientationPatient
    if FrameOfReferenceUID: dicom_file_data.FrameOfReferenceUID = FrameOfReferenceUID
    if PositionReferenceIndicator: dicom_file_data.PositionReferenceIndicator = PositionReferenceIndicator
    if SamplesPerPixel: dicom_file_data.SamplesPerPixel = SamplesPerPixel
    if PhotometricInterpretation: dicom_file_data.PhotometricInterpretation = PhotometricInterpretation
    if Rows: dicom_file_data.Rows = Rows
    if Columns: dicom_file_data.Columns = Columns
    if PixelSpacing: dicom_file_data.PixelSpacing = PixelSpacing
    if BitsAllocated: dicom_file_data.BitsAllocated = BitsAllocated
    if BitsStored: dicom_file_data.BitsStored = BitsStored
    if HighBit: dicom_file_data.HighBit = HighBit
    if PixelRepresentation: dicom_file_data.PixelRepresentation = PixelRepresentation
    if WindowCenter: dicom_file_data.WindowCenter = WindowCenter
    if WindowWidth: dicom_file_data.WindowWidth = WindowWidth
    if WindowWidth: dicom_file_data = WindowWidth
    if RescaleIntercept: dicom_file_data.RescaleIntercept = RescaleIntercept
    if RescaleType: dicom_file_data.RescaleType = RescaleType
    if PixelData: dicom_file_data.PixelData = PixelData

    return dicom_file_data

In [85]:
new_dicom_with_tags =  change_tags('data/slice.IMA', ImageType='RGB')
new_dicom_with_tags.save_as('data/edited_slice.IMA')

In [87]:
uploaded_dicom = pydicom.dcmread('data/edited_slice.IMA')
uploaded_dicom.ImageType

'RGB'

## Change the Pixel Array Value

In [88]:
uploaded_dicom.pixel_array

array([[972, 972, 972, ..., 971, 971, 971],
       [972, 973, 972, ..., 971, 971, 971],
       [972, 972, 972, ..., 970, 972, 972],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]],
      shape=(512, 512), dtype=uint16)

In [89]:
type(uploaded_dicom.PixelData)

bytes

In [90]:
new_array = np.zeros((512,512)).tobytes()

In [91]:
new_dicom_pixel_data = change_tags('data/slice.IMA', PixelData=new_array)

In [92]:
new_dicom_pixel_data.pixel_array

d:\anaconda\envs\dicom-env\Lib\site-packages\pydicom\pixels\decoders\base.py:801: UserWarning: The number of bytes of pixel data is sufficient to contain 4 frames which is larger than the given (0028,0008) 'Number of Frames' value of 1. The returned data will include these extra frames and if it's correct then you should update 'Number of Frames' accordingly, otherwise pass 'allow_excess_frames=False' to return only the first 1 frames.
  warn_and_log(


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], shape=(4, 512, 512), dtype=uint16)

## Convert Dicom into png / jpeg

In [94]:
array = normalize_visualize_dicom_2('data/slice.IMA', show=False)
img = Image.fromarray(array)
img.save('data/slice.png')

In [96]:
def dicom_to_png(dcm_file, png_path, max_v=None, min_v=None): # png/path/something.dcm
    slice_name = os.path.basename(dcm_file)[:-4]
    array = normalize_visualize_dicom_2(dcm_file, max_v, min_v, show=False)
    img = Image.fromarray(array)
    img.save(f'{png_path}/{slice_name}.png')

## Convert a group of Dicoms to png

In [97]:
from glob import glob

In [98]:
dicom_files_path = glob('multiple/*.IMA')

In [101]:
for dicom_file in dicom_files_path:
    dicom_to_png(dicom_file, 'multiple/multi_conv')

## Convert a group of dicom to nifty

In [105]:

import dicom2nifti


In [107]:

path_dicom_folder = 'multiple'
dicom2nifti.dicom_series_to_nifti(path_dicom_folder, 'multiple/multi_conv.nii.gz')

Removing duplicate slice from series
Removing duplicate slice from series


IndexError: list index out of range

## Convert a group to mp4


In [108]:
import cv2

In [109]:
dicom_files_path = sorted(glob('multiple/*.IMA'))
frame_per_second = 15
frame_size = pydicom.dcmread(dicom_files_path[0]).pixel_array.shape

In [110]:
out = cv2.VideoWriter('data/case_corona_scan.mp4', cv2.VideoWriter_fourcc(*'DIVX'), frame_per_second, frame_size)

In [112]:
for filename in dicom_files_path:
    img = normalize_visualize_dicom_2(filename, show=False)
    cv2_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    out.write(cv2_img)

out.release()